In [19]:
'''
    Importing necessary modules
'''
from keras.datasets import boston_housing
from keras import models
from keras import layers
import numpy as np

In [20]:
(train_data, train_target), (test_data, test_target) = boston_housing.load_data()

In [21]:
'''
    We will normalise all the data. Different ranges for different features not a good idea.
    Normalising involves subtracting the mean and dividing by the std
'''
mean = train_data.mean(axis = 0)
train_data = train_data - mean
std = train_data.std(axis = 0)
train_data = train_data/std

'''
    Note: for normalising test data, we use parameters from the training data - the mean and std
    of the training data
'''

test_data = test_data - mean
test_data = test_data/std

In [27]:
'''
    Model architecture
'''
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation = 'relu', input_shape = (13, )))
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(1)) #no activation function as we want the O/P to be linear. 
    model.compile(optimizer='rmsprop', loss = 'mse', metrics=['mae'])
    return model


In [28]:
'''
    K-folds cross validation and fitting the model. We are setting num_epochs to 100. Turns out there is quite
    a bit of fluctuation on cross validation scores. We will later increase the epochs to 500 to allow time
    to stabilise
'''
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = train_target[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(
    [train_data[:i * num_val_samples],
    train_data[(i + 1) * num_val_samples:]],
    axis=0)
    partial_train_targets = np.concatenate(
    [train_target[:i * num_val_samples],
    train_target[(i + 1) * num_val_samples:]],
    axis=0)
    model = build_model()
    model.fit(partial_train_data, partial_train_targets,
    epochs=num_epochs, batch_size=1, verbose=0)
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3


In [34]:
model = build_model()
model.fit(train_data, train_target,
epochs=80, batch_size=16, verbose=0)
test_mse_score, test_mae_score = model.evaluate(test_data, test_target)

102/102 [==============================] - 0s 147us/step


In [35]:
'''
    Final test scores - we are on average $2500 from the actual price
'''
test_mae_score

2.556213617324829